In [1]:
import os

In [2]:
!pwd

/c/Users/aeccs/Desktop/resume projects/End-To-End-Text-Summarization-Using-NLP/research


In [3]:
os.chdir("../")

In [4]:
!pwd

/c/Users/aeccs/Desktop/resume projects/End-To-End-Text-Summarization-Using-NLP


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
            url=self.config.source_URL,
            filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-17 18:58:21,590: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-17 18:58:21,591: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-17 18:58:21,595: INFO: common: created directory at: artifacts]
[2025-07-17 18:58:21,595: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-17 18:58:30,182: INFO: 4030192052: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 20718321
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "17d9bda3c61d5ebeb4739e54ee204dadc142bbbf302a4b8b485c89057ac7c357"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0EF5:3B1CA1:DFCD3:2074B6:6878FA75
Accept-Ranges: bytes
Date: Thu, 17 Jul 2025 13:28:24 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830037-CCU
X-Cache: 